In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import operator
import pandas as pd
import re
import seaborn as sns

from classifier import *
from heapq import nlargest
from issues import get_num_code_lines
from nltk.stem import PorterStemmer
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Pre-Processing

In [2]:
#TODO: concat all these into one file
with open('../data/eclipse/eclpise_issues1.json') as json_data:
    issues1 = json.load(json_data)
with open('../data/eclipse/eclpise_issues2.json') as json_data:
    issues2 = json.load(json_data)
with open('../data/eclipse/eclpise_issues3.json') as json_data:
    issues3 = json.load(json_data)
with open('../data/eclipse/eclpise_issues4.json') as json_data:
    issues4 = json.load(json_data)
with open('../data/eclipse/eclpise_issues5.json') as json_data:
    issues5 = json.load(json_data)
with open('../data/eclipse/eclpise_issues6.json') as json_data:
    issues6 = json.load(json_data)
with open('../data/eclipse/eclpise_issues7.json') as json_data:
    issues7 = json.load(json_data)
with open('../data/eclipse/eclpise_issues8.json') as json_data:
    issues8 = json.load(json_data)
with open('../data/eclipse/eclpise_issues9.json') as json_data:
    issues9 = json.load(json_data)
with open('../data/eclipse/eclpise_issues10.json') as json_data:
    issues10 = json.load(json_data)
with open('../data/eclipse/eclpise_issues11.json') as json_data:
    issues11 = json.load(json_data)
issues = issues1 + issues2 + issues3 + issues4 + issues5 + issues6 + issues7 + issues8 + issues9 + issues10 + issues11

In [3]:
print("Number of issues: " + str(len(issues)))
labeled_issues = [issue for issue in issues if issue['completed_by']]
print("Number of labeled issues: " + str(len(labeled_issues)))

Number of issues: 173401
Number of labeled issues: 88536


In [4]:
# create a data frame from the list of issues
df_list = []
for issue in labeled_issues[25:]:
        df_dict = {}
        df_dict['title'] = issue['short_desc'].replace("\n", "").replace("\t", "")
        df_dict['body'] = issue['long_desc'].replace("\n", "").replace("\t", "")
        df_dict['closed_date'] = pd.to_datetime(issue['completed_at'][:10])
        df_dict['created_date'] = pd.to_datetime(issue['created_at'][:10])
        df_dict['completed_by'] = issue['completed_by']
        df_dict['product'] = issue['product']
        df_dict['component'] = issue['component']
        df_list.append(df_dict)
df = pd.DataFrame(df_list).sort_values('closed_date')
df.head(2)

,body,closed_date,completed_by,component,created_date,product,title
392,"I saw this happen on linux, but I'm sure it ha...",2001-10-11,unknown,UI\n\n (show other bugs)\n,2001-10-10,Platform\n\n,[UI] Unzoom occurs if you select file in navig...
251,DS (10/9/01 4:16:49 PM)Create two classesActio...,2001-10-11,jared_burns,Debug\n\n (show other bugs)\n,2001-10-10,JDT\n\n,"Breakpoints installed in all classes with ""nam..."


In [5]:
# one hot encode the component and team columns
df = pd.get_dummies(df, columns=["component", "product"], prefix=["component", "product"])
df.head(2)

,body,closed_date,completed_by,created_date,title,component_AJBrowser (show other bugs),component_AJDoc (show other bugs),component_ALF (show other bugs),component_AM3(deprecated) (show other bugs),component_AMW (show other bugs),...,product_WTP EJB Tools,product_WTP Incubator,product_WTP Java EE Tools,product_WTP Releng,product_WTP ServerTools,product_WTP Source Editing,product_WTP Webservices,product_WTP Website,product_Web Tools,product_z_Archived
392,"I saw this happen on linux, but I'm sure it ha...",2001-10-11,unknown,2001-10-10,[UI] Unzoom occurs if you select file in navig...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
251,DS (10/9/01 4:16:49 PM)Create two classesActio...,2001-10-11,jared_burns,2001-10-10,"Breakpoints installed in all classes with ""nam...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# get list of devs who solve an issue in last three months (from end of train set)
active_devs = set()
dev_counts = {}
filter_date = df.iloc[50000]['closed_date'] - pd.to_timedelta(90, unit='d')
for _, row in df.iterrows():
    if (row['closed_date'] > filter_date):
        dev_counts[row['completed_by']] = dev_counts.get(row['completed_by'], 0) + 1
#print(dev_counts)
for dev in dev_counts:
    if dev_counts[dev] >= 3:
        active_devs.add(dev)
#print(active_devs)
# remove all issues not solved by an active dev
df = df[df['completed_by'].isin(active_devs)].reset_index(drop=True)
print(len(df))

77704


In [7]:
def preprocess(text):
    stemmer = PorterStemmer()
    # remove punctuation and non-alpha numeric characters
    split1 = ' '.join([word for word in re.split('\W+', text) if word.isalpha()])
    # split camel case words apart (necessary for embedded code) and apply stemmer to all words
    split2 = ' '.join([stemmer.stem(word) for word in re.sub('(?!^)([A-Z][a-z]+)', r' \1', split1).split()])
    return split2

In [8]:
# use tf-idf w/ stemming, stop-word removal, and non-alphabetic word removal to generate features
df['body'] = df['body'].apply(preprocess)
vectorizer_body = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)
vectorizer_body.fit(df['body'])
vector_body = vectorizer_body.transform(df['body'])
# summarize encoded vector
print(vector_body.shape)
df['title'] = df['title'].apply(preprocess)
vectorizer_title = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)
vectorizer_title.fit(df['title'])
vector_title = vectorizer_title.transform(df['title'])
# summarize encoded vector
print(vector_title.shape)

(77704, 69040)
(77704, 9049)


In [9]:
print(type(vector_title))
print(type(vector_body))
sparse_data = hstack((vector_title, vector_body))
print(sparse_data.shape)
#TODO: these should be numerical features before combining
df['completed_by'] = df['completed_by'].astype('category')
df['completed_by_encode'] = df['completed_by'].cat.codes
#df['completed_by'] = pd.factorize(df['completed_by'])
#sparse_data = hstack((sparse_data,np.array(df['completed_by_encode'])[:,None]))
print(sparse_data.shape)
#TODO: add component and team
filter_cols = [col for col in df if col.startswith('product') or col.startswith('component')]
sparse_data = hstack((sparse_data,np.array(df[filter_cols]))).tocsr()
print(sparse_data.shape)

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
(77704, 78089)
(77704, 78089)
(77704, 78417)


# Machine Learning

In [10]:
size = 75000
X_train = sparse_data[:size]
X_test = sparse_data[size:]
y_train = df['completed_by'][:size]
y_test = df['completed_by'][size:]

'''
train_df = df[:size]
test_df = df[size:]
X_train = train_df.drop(['body', 'closed_date', 'completed_by', 'created_date', 'title', 'completed_by_encode'], axis=1)
y_train = train_df['completed_by']
X_test = test_df.drop(['body', 'closed_date', 'completed_by', 'created_date', 'title', 'completed_by_encode'], axis=1)
y_test = test_df['completed_by']
'''
#print(len(X_train))
#print(len(X_test))
#X_train.head(5)

"\ntrain_df = df[:size]\ntest_df = df[size:]\nX_train = train_df.drop(['body', 'closed_date', 'completed_by', 'created_date', 'title', 'completed_by_encode'], axis=1)\ny_train = train_df['completed_by']\nX_test = test_df.drop(['body', 'closed_date', 'completed_by', 'created_date', 'title', 'completed_by_encode'], axis=1)\ny_test = test_df['completed_by']\n"

In [11]:
def is_correct_top_k(pred_prob, k, actual, labels):
    indices = [i for i in range(len(pred_prob))]
    top_indices = nlargest(k, indices, key=lambda i: pred_prob[i])
    top_choices = set([labels[i] for i in top_indices])
    return actual in top_choices

In [12]:
classifier = MLPClassifier(hidden_layer_sizes=(25,25))
classifier.fit(X_train,y_train)
classes = classifier.classes_
print("Top 1 Accuracy: " + str(classifier.score(X_test, y_test)))
pred_probs = classifier.predict_proba(X_test)
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 2, y_test[size + idx], classes):
        correct += 1
print ("Top 2 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 3, y_test[size + idx], classes):
        correct += 1
print ("Top 3 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 4, y_test[size + idx], classes):
        correct += 1
print ("Top 4 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 5, y_test[size + idx], classes):
        correct += 1
print ("Top 5 Accuracy: " + str((correct/len(y_test))))

Top 1 Accuracy: 0.23631656804733728
Top 2 Accuracy: 0.35022189349112426
Top 3 Accuracy: 0.4145710059171598
Top 4 Accuracy: 0.4681952662721893
Top 5 Accuracy: 0.5077662721893491


In [13]:
'''
classifier = GaussianNB()
classifier.fit(X_train,y_train)
classes = classifier.classes_
print("Top 1 Accuracy: " + str(classifier.score(X_test, y_test)))
pred_probs = classifier.predict_proba(X_test)
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 2, y_test[size + idx], classes):
        correct += 1
print ("Top 2 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 3, y_test[size + idx], classes):
        correct += 1
print ("Top 3 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 4, y_test[size + idx], classes):
        correct += 1
print ("Top 4 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 5, y_test[size + idx], classes):
        correct += 1
print ("Top 5 Accuracy: " + str((correct/len(y_test))))
'''

'\nclassifier = GaussianNB()\nclassifier.fit(X_train,y_train)\nclasses = classifier.classes_\nprint("Top 1 Accuracy: " + str(classifier.score(X_test, y_test)))\npred_probs = classifier.predict_proba(X_test)\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 2, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 2 Accuracy: " + str((correct/len(y_test))))\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 3, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 3 Accuracy: " + str((correct/len(y_test))))\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 4, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 4 Accuracy: " + str((correct/len(y_test))))\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 5, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 5 Accuracy: " + str((correct/len(y_test))))\n'

In [14]:
'''
classifier = BernoulliNB()
classifier.fit(X_train,y_train)
classes = classifier.classes_
print("Top 1 Accuracy: " + str(classifier.score(X_test, y_test)))
pred_probs = classifier.predict_proba(X_test)
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 2, y_test[size + idx], classes):
        correct += 1
print ("Top 2 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 3, y_test[size + idx], classes):
        correct += 1
print ("Top 3 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 4, y_test[size + idx], classes):
        correct += 1
print ("Top 4 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 5, y_test[size + idx], classes):
        correct += 1
print ("Top 5 Accuracy: " + str((correct/len(y_test))))
'''

'\nclassifier = BernoulliNB()\nclassifier.fit(X_train,y_train)\nclasses = classifier.classes_\nprint("Top 1 Accuracy: " + str(classifier.score(X_test, y_test)))\npred_probs = classifier.predict_proba(X_test)\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 2, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 2 Accuracy: " + str((correct/len(y_test))))\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 3, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 3 Accuracy: " + str((correct/len(y_test))))\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 4, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 4 Accuracy: " + str((correct/len(y_test))))\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 5, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 5 Accuracy: " + str((correct/len(y_test))))\n'

In [15]:
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train, y_train)
classes = classifier.classes_
print("Top 1 Accuracy: " + str(classifier.score(X_test, y_test)))
pred_probs = classifier.predict_proba(X_test)
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 2, y_test[size + idx], classes):
        correct += 1
print ("Top 2 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 3, y_test[size + idx], classes):
        correct += 1
print ("Top 3 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 4, y_test[size + idx], classes):
        correct += 1
print ("Top 4 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 5, y_test[size + idx], classes):
        correct += 1
print ("Top 5 Accuracy: " + str((correct/len(y_test))))

Top 1 Accuracy: 0.2703402366863905
Top 2 Accuracy: 0.3779585798816568
Top 3 Accuracy: 0.4145710059171598
Top 4 Accuracy: 0.41494082840236685
Top 5 Accuracy: 0.41494082840236685


In [16]:
'''
classifier = RandomForestClassifier(n_estimators=1000)
classifier.fit(X_train, y_train)
classes = classifier.classes_
print("Top 1 Accuracy: " + str(classifier.score(X_test, y_test)))
pred_probs = classifier.predict_proba(X_test)
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 2, y_test[size + idx], classes):
        correct += 1
print ("Top 2 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 3, y_test[size + idx], classes):
        correct += 1
print ("Top 3 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 4, y_test[size + idx], classes):
        correct += 1
print ("Top 4 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 5, y_test[size + idx], classes):
        correct += 1
print ("Top 5 Accuracy: " + str((correct/len(y_test))))
'''

'\nclassifier = RandomForestClassifier(n_estimators=1000)\nclassifier.fit(X_train, y_train)\nclasses = classifier.classes_\nprint("Top 1 Accuracy: " + str(classifier.score(X_test, y_test)))\npred_probs = classifier.predict_proba(X_test)\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 2, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 2 Accuracy: " + str((correct/len(y_test))))\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 3, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 3 Accuracy: " + str((correct/len(y_test))))\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 4, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 4 Accuracy: " + str((correct/len(y_test))))\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 5, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 5 Accuracy: " + str

In [17]:
#TODO: try SVC (might need to do something else)
'''
classifier = SVC(probability=True)
classifier.fit(X_train, y_train)
classes = classifier.classes_
print("Top 1 Accuracy: " + str(classifier.score(X_test, y_test)))
pred_probs = classifier.predict_proba(X_test)
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 2, y_test[size + idx], classes):
        correct += 1
print ("Top 2 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 3, y_test[size + idx], classes):
        correct += 1
print ("Top 3 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 4, y_test[size + idx], classes):
        correct += 1
print ("Top 4 Accuracy: " + str((correct/len(y_test))))
correct = 0
for idx, pred in enumerate(pred_probs):
    if is_correct_top_k(pred, 5, y_test[size + idx], classes):
        correct += 1
print ("Top 5 Accuracy: " + str((correct/len(y_test))))
'''

'\nclassifier = SVC(probability=True)\nclassifier.fit(X_train, y_train)\nclasses = classifier.classes_\nprint("Top 1 Accuracy: " + str(classifier.score(X_test, y_test)))\npred_probs = classifier.predict_proba(X_test)\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 2, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 2 Accuracy: " + str((correct/len(y_test))))\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 3, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 3 Accuracy: " + str((correct/len(y_test))))\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 4, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 4 Accuracy: " + str((correct/len(y_test))))\ncorrect = 0\nfor idx, pred in enumerate(pred_probs):\n    if is_correct_top_k(pred, 5, y_test[size + idx], classes):\n        correct += 1\nprint ("Top 5 Accuracy: " + str((correct/len(y_test

# Analysis

In [18]:
classifier = MLPClassifier(hidden_layer_sizes=(25,25))
classifier.fit(X_train,y_train)
preds = classifier.predict(X_test)
print(preds)

['snorthov' 'mschorn.eclipse' 'ccc' ... 'Tod_Creasey' 'lynne_kues'
 'eclipse']


/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [19]:
actual = test_df['completed_by']
pd.value_counts(actual).plot.bar()

NameError: name 'test_df' is not defined

In [ ]:
pd.value_counts(df['completed_by']).plot.bar()

In [ ]:
pd.value_counts(preds).plot.bar()

In [ ]:
overall_counts = pd.value_counts(df['completed_by'])
# list of devs sorted in order of highest contribution
sorted_dev_list = overall_counts.index.values
pred_counts = pd.value_counts(preds)
actual_counts = pd.value_counts(actual)

percent_diff = {}
vals = []
counts = []
missing = set()
for dev in sorted_dev_list:
    if dev in pred_counts:
        val = 100 * (pred_counts[dev] - actual_counts[dev]) / (actual_counts[dev])
        percent_diff[dev] = val
        counts.append(overall_counts[dev])
        vals.append(val)
    else:
        missing.add(dev)
print("devs not included in predictions: ")
print(list(missing))

In [ ]:
plt.bar(percent_diff.keys(), percent_diff.values())
plt.title("Percent Difference Prediction Rate vs. Actual Rate")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# remove the outlier
del vals[2]
del counts[2]
# create best fit line
z = np.polyfit(x=counts, y=vals, deg=1)
p = np.poly1d(z)
trend_line = p(counts)
# test best fit
yhat = trend_line    
ybar = np.sum(vals)/len(vals)
ssreg = np.sum((yhat-ybar)**2) 
sstot = np.sum((vals - ybar)**2)
print("R^2: " + str(ssreg / sstot))
# create plots
plt.scatter(counts, vals)
plt.title("Percent Difference Between Prediction Rate and Actual Rate vs. Issue Count")
plt.xticks(rotation=90)
plt.xlabel("Number of Issues Solved")
plt.ylabel("% diff. pred rate and actual rate")
plt.plot(counts, trend_line)
plt.show()

In [ ]:
#TODO: look at assigning open issues and seeing what overspecialization problem would look like.

In [ ]:
test_df.head(5)

In [ ]:
test_df.tail(5)